## Introduction

### Import libs

In [ ]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline 

import warnings
warnings.filterwarnings("ignore")

### Read Data 

In [ ]:
corona_brazil = pd.read_csv("../input/corona-virus-brazil/brazil_covid19.csv")

In [ ]:
last_day = list(corona_brazil['date'])[-1]

### Knowing the database

In [ ]:
corona_brazil.info()

In [ ]:
corona_brazil.columns

### Removendo valores duplicados

In [ ]:
corona_brazil[corona_brazil['date']=='2020-03-22']['cases'].sum()

In [ ]:
corona_brazil[corona_brazil['date']=='2020-03-23']['cases'].sum()

In [ ]:
corona_brazil['chave'] = corona_brazil.apply(lambda row: row['date']+row['region']+row['state']+str(row['cases'])+str(row['deaths']), axis=1)

In [ ]:
corona_brazil = corona_brazil.drop_duplicates(['chave'], keep='last' )

## Analyzing the Data

### Creating a function for analyz evolution per day

In [ ]:
def acumulado(infectado):
    if len(lista_infectado)>0:
        infectado = infectado + lista_infectado[-1]
        lista_infectado.append(infectado)
    return infectado

### Column with numbers acumulated

In [ ]:
corona_sp = corona_brazil[corona_brazil['state']=="São Paulo"]

## Create a analyz per day of cases

In [ ]:
def analise_dia(dia, what):
    infectados_dia = corona_brazil[corona_brazil['date']==dia][what].sum()
    return infectados_dia

In [ ]:
corona_dia = pd.DataFrame()
corona_dia['date'] = corona_brazil['date'].unique()

In [ ]:
corona_dia['Infectados_acumulado'] = corona_dia.apply(lambda row: analise_dia(row['date'], 'cases'), axis=1)
corona_dia['Mortos_acumulado'] = corona_dia.apply(lambda row: analise_dia(row['date'], 'deaths'), axis=1)

### Evolution of cases acumulated

In [ ]:
def casos_dia(row):
    casos = row-lista_casos[-1]
    lista_casos.append(row)
    return casos

In [ ]:
lista_casos = [0]
corona_sp['cases_day'] = corona_sp.apply(lambda row: casos_dia(row['cases']), axis=1 )

In [ ]:
lista_casos = [0]
corona_sp['deaths_day'] = corona_sp.apply(lambda row: casos_dia(row['deaths']), axis=1 )

In [ ]:
lista_casos = [0]
corona_dia['cases_day'] = corona_dia.apply(lambda row: casos_dia(row['Infectados_acumulado']), axis=1 )

In [ ]:
lista_casos = [0]
corona_dia['death_day'] = corona_dia.apply(lambda row: casos_dia(row['Mortos_acumulado']), axis=1 )

### São Paulo

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(x='date', y='cases_day', data=corona_sp,palette = "Blues")
plt.xticks(rotation=-90)
for ponto in list(corona_sp.index):
    plt.text(x=corona_sp['date'][ponto], y=corona_sp['cases_day'][ponto], s=corona_sp['cases_day'][ponto], fontsize=15)
plt.title('Gráfico Evolução de Infectados por dia em São Paulo')

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x='date', y='cases', data=corona_sp,palette = "Blues")
plt.xticks(rotation=-90)
plt.title('Gráfico Evolução de Infectados Acumulado em São Paulo')

### Brazil

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(x='date', y='cases_day', data=corona_dia,palette = "Blues")
plt.xticks(rotation=-90)
for ponto in list(corona_dia.index):
    plt.text(x=corona_dia['date'][ponto], y=corona_dia['cases_day'][ponto], s=corona_dia['cases_day'][ponto], fontsize=15)
plt.title('Gráfico Evolução de Infectados por dia no Brasil')

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x='date', y='Infectados_acumulado', data=corona_dia,palette = "Blues")
plt.xticks(rotation=-90)
plt.title('Gráfico Evolução de Infectados Acumulado no Brasil')

### Cases per state

#### Creating dataframe clustered per states

In [ ]:
estados = list(corona_brazil['state'].unique())
corona_estado = pd.DataFrame(columns=['Estado','Suspeitos','Infectados','Liberados','Mortos'])

In [ ]:
Infectados = []
Mortos = []
corona_filter = corona_brazil[corona_brazil['date']==last_day]
for estado in estados:    
    Infectados.append(corona_filter[corona_filter['state']==estado]['cases'].sum())
    Mortos.append(corona_filter[corona_filter['state']==estado]['deaths'].sum())
corona_estado['Estado'] = estados
corona_estado['Infectados'] = Infectados
corona_estado['Mortos'] = Mortos

#### Analyz of cases per state

In [ ]:
ordernation = corona_estado.groupby(["Estado"])['Infectados'].aggregate(np.median).reset_index().sort_values('Infectados')

order = []
lista_estados_ord = list(ordernation['Estado'])
contagem = len(lista_estados_ord)

for i in range(contagem-1, -1, -1):
    order.append(lista_estados_ord[i])
    

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(x="Infectados", y="Estado", data=corona_estado, color="b", order=order)
print('Gráfico de casos por estados')

### Deaths per state

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(x='date', y='deaths_day', data=corona_sp,palette = "Blues")
plt.xticks(rotation=-90)
for ponto in list(corona_sp.index):
    plt.text(x=corona_sp['date'][ponto], y=corona_sp['deaths_day'][ponto], s=corona_sp['deaths_day'][ponto], fontsize=15)
plt.title('Gráfico Evolução de Mortos por dia em São Paulo')

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(x="Mortos", y="Estado", data=corona_estado, color="b", order=order)
plt.title('Gráfico de mortes por estados')

### Evolution deaths in Brazil

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(x='date', y='death_day', data=corona_dia,palette = "Blues")
plt.xticks(rotation=-90)
for ponto in list(corona_dia.index):
    plt.text(x=corona_dia['date'][ponto], y=corona_dia['death_day'][ponto], s=corona_dia['death_day'][ponto], fontsize=15)
plt.title('Gráfico Evolução de Mortos por dia no Brasil')

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x='date', y='Mortos_acumulado', data=corona_dia,palette = "Blues")
plt.xticks(rotation=-90)
plt.title('Gráfico Evolução de Mortos Acumulado no Brasil')

## Conclusion

In this analysis I conclude that Brazil is evolving in the same way as other countries. The growth is quite exponential and the capitals (where we have a higher rate of people traveling to other countries) are the places with the highest rate of evolution of the disease.
I discovery a error in padronization of data base. The cases and deaths dont should be sum, different from suspects and refuses, that the data is refresh per day.